In [29]:
import pandas as pd
import plotly.express as px
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima.model import ARIMA

# Daten laden und vorbereiten
def daten_laden(datei_pfad, datum_spalte, wert_spalte):
    daten = pd.read_csv(datei_pfad, parse_dates=[datum_spalte])
    daten.set_index(datum_spalte, inplace=True)
    daten = daten[[wert_spalte]].dropna()
    return daten

# Trend und Saisonalität entfernen
def trend_und_saisonalität_entfernen(daten, wert_spalte, frequenz):
    dekomposition = seasonal_decompose(daten[wert_spalte], period=frequenz)
    daten["Trend"] = dekomposition.trend
    daten["Saisonalität"] = dekomposition.seasonal
    daten["Residuen"] = dekomposition.resid
    return daten, dekomposition

# ARIMA mit Update-Funktion anwenden und Vorhersagen erzeugen
def arima_mit_update(daten, trainingsanteil=0.7):
    # Aufteilen in Trainings- und Testdaten
    trainingsgröße = int(len(daten) * trainingsanteil)
    daten_training, daten_test = daten.iloc[:trainingsgröße], daten.iloc[trainingsgröße:]

    # ARIMA-Modell trainieren
    modell = ARIMA(daten_training["Residuen"].dropna(), order=(1, 1, 1))
    trainiertes_modell = modell.fit()

    # Vorhersagen für den Testzeitraum berechnen
    vorhersagen = trainiertes_modell.forecast(steps=len(daten_test))  # Vorhersagen für den gesamten Testzeitraum

    return daten_training, daten_test, vorhersagen

# Ergebnisse grafisch darstellen
def ergebnisse_darstellen_gesamtzeitraum(daten, daten_training, daten_test, vorhersagen):
    # Vorhersagen für die Testdaten berechnen und in den Testdaten einfügen
    daten_test["Vorhersagen"] = vorhersagen + daten_test["Trend"].iloc[:len(vorhersagen)] + daten_test["Saisonalität"].iloc[:len(vorhersagen)]

    # Linienplot
    fig = px.line()

    # Trainingsdaten blau darstellen
    fig.add_scatter(x=daten_training.index, y=daten_training["Consumption"], mode='lines', name='Trainingsdaten (Original)', line=dict(color='blue'))

    # Testdaten (letzte 10%) in roter Farbe darstellen
    fig.add_scatter(x=daten_test.index, y=daten_test["Consumption"], mode='lines', name='Testdaten (Original)', line=dict(color='red'))

    # Vorhersagen für die Testdaten in orange darstellen
    fig.add_scatter(x=daten_test.index, y=daten_test["Vorhersagen"], mode='lines', name='Testdaten (Vorhersage)', line=dict(color='orange'))

    # Visualisierung optimieren
    fig.update_layout(
        title="Trainingsdaten, Testdaten und Vorhersagen",
        xaxis_title="Datum",
        yaxis_title="Energieverbrauch",
        legend_title="Legende"
    )
    
    # Anzeige der Grafik
    fig.show()

# Hauptprogramm
if __name__ == "__main__":
    datei_pfad = "opsd_germany_daily.csv"  # Pfad zur CSV-Datei
    datum_spalte = "Date"        # Spalte mit Datumswerten
    wert_spalte = "Consumption"  # Spalte mit Verbrauchswerten

    # Daten laden
    daten = daten_laden(datei_pfad, datum_spalte, wert_spalte)

    # Trend und Saisonalität entfernen
    daten, dekomposition = trend_und_saisonalität_entfernen(daten, wert_spalte, frequenz=365)

    # ARIMA mit Update anwenden
    daten_training, daten_test, vorhersagen = arima_mit_update(daten)

    # Ergebnisse im Gesamtzeitraum darstellen
    ergebnisse_darstellen_gesamtzeitraum(daten, daten_training, daten_test, vorhersagen)


D:\Lehre\Zeitreihen_Analyse_und_Prädiktion\venv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning:

No frequency information was provided, so inferred frequency D will be used.

D:\Lehre\Zeitreihen_Analyse_und_Prädiktion\venv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning:

No frequency information was provided, so inferred frequency D will be used.

D:\Lehre\Zeitreihen_Analyse_und_Prädiktion\venv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning:

No frequency information was provided, so inferred frequency D will be used.

C:\Users\torsten.schmidt\AppData\Local\Temp\ipykernel_17712\918841390.py:39: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

